# Import Required Libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


We would also be needing a couple of packages for our project. Namely, Geocoders and Folium for the purpose of converting an address into latitude and longitude values and the latter for plotting geospatial data.

Geocoders: A package that converts a give address into Latitude and Longitude values
Folium: A very useful package for plotting Geospatial data

In [3]:
!conda install -c conda-forge geopy --yes

Solving environment: - ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt



In [6]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Install folium Library & Import Folium Module

In [7]:
pip install folium

     |████████████████████████████████| 93 kB 3.9 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import folium

# Download the dataset

we will download the New Your City's dataset from the IBM Cloude which we used in Week 3 of this Course
We will be using the !wget command to downoad the JSON file

In [9]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


Load the data

In [10]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's scrap the 'Features' data from the JSON file and load in a new dataset called 'neighborhoods_data'

In [11]:
neighborhoods_data = newyork_data['features']

Lets have a look at the first record of our new data

In [12]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

# Transform data into Pandas dataframe

Create and empty dataframe

In [17]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Now fill in our empty Pandas dataframe from our JSON format data

In [18]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [19]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


# Get the Longitude and Latitude of New York City

In [24]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


# # Create a Map of New York City with Neighborhoods

In [25]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Segment the neighborhoods of all Boroughs of New York

Now, from here we will simplify the above map and segment and cluster neighborhoods of one borough at a time. There are 5 boroughs in New York; Brooklyn, Manhattan, Queens, Bronx and Staten Island. We will segment and cluster neighborhoods of all 5 boroughs, starting with Brooklyn. 

Slice the original dataframe and create a new datframe of Brooklyn

In [26]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


Get the Geographical coordinates of Brooklyn

In [27]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


Lets Visualize the Brooklyn and its neighborhoods

In [28]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

Slice the original dataframe and create a dataframe for Manhattan

In [29]:
Manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
Manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Get the Geographical coordinates of Manhattan

In [38]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


Visualize the Manhattan and its Neighborhoods

In [184]:

# create map of Manhattan using latitude2 and longitude2 values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Manhattan_data['Latitude'], Manhattan_data['Longitude'], Manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Slice the original dataframe and create a dataframe for Queens

In [35]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


Get the Geographical coordinates of Queens

In [37]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


Visualize the Queens and its neighborhoods on a folium map

In [46]:
# create map of Manhattan using latitude2 and longitude2 values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

Slice the original dataframe and create a dataframe for Bronx

In [41]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Get geographical coordinates of Bronx

In [42]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.8506558, -73.8665241.


Visualize Bronx and its neighborhoods 

In [47]:
# create map of Manhattan using latitude2 and longitude2 values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

Slice the original dataframe and create a new dataframe of Staten Island

In [48]:
staten_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
staten_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


Get the geographical coordinates of Staten Island

In [49]:
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.5834557, -74.1496048.


Visualize the Staten Island and its Neighborhoods

In [50]:
# create map of Manhattan using latitude2 and longitude2 values
map_staten = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(staten_data['Latitude'], staten_data['Longitude'], staten_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_staten)  
    
map_staten

## Now, we will utilizing the Foursquare API to explore the neighborhoods and segment them. 

First define your Foursquare Credentials

In [51]:
CLIENT_ID = 'YOIIQGRRGN5CHG5VFRTX5SFMKUS1S41LXLRJTQIXO1LOIJL0'
CLIENT_SECRET = '1LSJDTVXYTRUDWNTS21K3NYUYSK5SLTTFXG34N0ELQL000SH'
VERSION = '20180605'

# Now we will explore all our 5 newly created datasets one by one

Lets explore the first neighorhood in our Brooklyn dataset

In [53]:
brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

Get the Long and Lat of the neighborhood

In [54]:
neighborhood_latitude = brooklyn_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = brooklyn_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = brooklyn_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


Create a Get Request URL to get top 100 venues within the 500 meter radius of Bay Ridge

In [56]:
# limit of number of venues returned by Foursquare API
LIMIT = 100

# define radius
radius = 500

# URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb581c5368fe872252d5d3f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 83,
  'suggestedBounds': {'ne': {'lat': 40.63030106951066,
    'lng': -74.02470273356597},
   'sw': {'lat': 40.62130106051065, 'lng': -74.03653865351028}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b895827f964a5206c2d32e3',
       'name': 'Pilo Arts Day Spa and Salon',
       'location': {'address': '8412 3rd Ave',
        'lat': 40.62474788273414,
        'lng': -74.03059056940135,
        'labeledLatL

Create get_category_type function

In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now Clean the results JSON and structure it into pandas dataframe

In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Georgian Dream Cafe and Bakery,Caucasian Restaurant,40.625586,-74.030196
3,Leo's Casa Calamari,Pizza Place,40.624200,-74.030931
4,Cocoa Grinder,Juice Bar,40.623967,-74.030863


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

83 venues were returned by Foursquare.


Lets do the same operation with our Manhatten Data

Get the Lat and Lng for first neighborhood of Manhattan

In [63]:
neighborhood_latitude2 = Manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude2 = Manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name2 = Manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name2, 
                                                               neighborhood_latitude2, 
                                                               neighborhood_longitude2))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [65]:

# limit of number of venues returned by Foursquare API
LIMIT = 100

# define radius
radius = 500

# URL
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude2, neighborhood_longitude2, radius, LIMIT)

url2
results2 = requests.get(url2).json()

In [66]:
venues2 = results2['response']['groups'][0]['items']
    
nearby_venues2 = json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues2 =nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

nearby_venues2.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


Do the same operation with Queens data's first neighborhood

In [67]:
neighborhood_latitude3 = queens_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude3 = queens_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name3 = queens_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name3, 
                                                               neighborhood_latitude3, 
                                                               neighborhood_longitude3))

Latitude and longitude values of Astoria are 40.76850859335492, -73.91565374304234.


In [68]:
# limit of number of venues returned by Foursquare API
LIMIT = 100

# define radius
radius = 500

# URL
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude3, neighborhood_longitude3, radius, LIMIT)

url3

results3 = requests.get(url3).json()

In [69]:
venues3 = results3['response']['groups'][0]['items']
    
nearby_venues3 = json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues3 =nearby_venues3.loc[:, filtered_columns3]

# filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]

nearby_venues3.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Favela Grill,Brazilian Restaurant,40.767348,-73.917897
1,Orange Blossom,Gourmet Shop,40.769856,-73.917012
2,CrossFit Queens,Gym,40.769404,-73.918977
3,Titan Foods Inc.,Gourmet Shop,40.769198,-73.919253
4,Off The Hook,Seafood Restaurant,40.767200,-73.918104


Same Operation needs to be done with Bronx data

In [70]:
neighborhood_latitude4 = bronx_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude4 = bronx_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name4 = bronx_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name4, 
                                                               neighborhood_latitude4, 
                                                               neighborhood_longitude4))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [71]:
# limit of number of venues returned by Foursquare API
LIMIT = 100

# define radius
radius = 500

# URL
url4 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude4, neighborhood_longitude4, radius, LIMIT)

results4 = requests.get(url4).json()
results4

{'meta': {'code': 200, 'requestId': '5fb585f14360832ce8d8d5c4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wakefield',
  'headerFullLocation': 'Wakefield, Bronx',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 40.899205181110005,
    'lng': -73.84125857127495},
   'sw': {'lat': 40.89020517211, 'lng': -73.8531424698231}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c537892fd2ea593cb077a28',
       'name': 'Lollipops Gelato',
       'location': {'address': '4120 Baychester Ave',
        'crossStreet': 'Edenwald & Bussing Ave',
        'lat': 40.894123150205274,
        'lng': -73.84589162362325,
        'labeledLatL

In [72]:

venues4 = results4['response']['groups'][0]['items']
    
nearby_venues4 = json_normalize(venues4) # flatten JSON

# filter columns
filtered_columns4 = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues4 =nearby_venues4.loc[:, filtered_columns4]

# filter the category for each row
nearby_venues4['venue.categories'] = nearby_venues4.apply(get_category_type, axis=1)

# clean columns
nearby_venues4.columns = [col.split(".")[-1] for col in nearby_venues4.columns]

nearby_venues4.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Rite Aid,Pharmacy,40.896649,-73.844846
2,Walgreens,Pharmacy,40.896528,-73.844700
3,Dunkin',Donut Shop,40.890459,-73.849089
4,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568


Same operation to be done with Staten Island

In [73]:
neighborhood_latitude5 = staten_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude5 = staten_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name5 = staten_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name5, 
                                                               neighborhood_latitude5, 
                                                               neighborhood_longitude5))

Latitude and longitude values of St. George are 40.6449815710044, -74.07935312512797.


In [74]:

# limit of number of venues returned by Foursquare API
LIMIT = 100

# define radius
radius = 500

# URL
url5 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude5, neighborhood_longitude5, radius, LIMIT)

results5 = requests.get(url5).json()
results5

{'meta': {'code': 200, 'requestId': '5fb5866e7694ba313b04c34f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 35,
  'suggestedBounds': {'ne': {'lat': 40.6494815755044,
    'lng': -74.07343346476772},
   'sw': {'lat': 40.6404815665044, 'lng': -74.08527278548821}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a214841f964a520cd7c1fe3',
       'name': 'Beso',
       'location': {'address': '11 Schuyler St',
        'crossStreet': 'btwn Richmond Terrace & Stuyvesant Pl',
        'lat': 40.64330638739738,
        'lng': 

In [75]:
venues5 = results5['response']['groups'][0]['items']
    
nearby_venues5 = json_normalize(venues5) # flatten JSON

# filter columns
filtered_columns5 = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues5 =nearby_venues5.loc[:, filtered_columns5]

# filter the category for each row
nearby_venues5['venue.categories'] = nearby_venues5.apply(get_category_type, axis=1)

# clean columns
nearby_venues5.columns = [col.split(".")[-1] for col in nearby_venues5.columns]

nearby_venues5.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Beso,Tapas Restaurant,40.643306,-74.076508
1,Staten Island September 11 Memorial,Monument / Landmark,40.646767,-74.076510
2,Richmond County Bank Ballpark,Baseball Stadium,40.645056,-74.076864
3,A&S Pizzeria,Pizza Place,40.643940,-74.077626
4,Shake Shack,Burger Joint,40.643660,-74.075891


# Lets Create a function to repeat the same process for all the neighborhoods of all Boroughs

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now pass brooklyn data in above the function getNearbyVenues to get all the venues of brooklyn 

In [86]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


lets check how many venues are returned for each neighborhood

In [87]:
brooklyn_venues.groupby('Neighborhood')[['Venue Category']].count()

,Venue Category
Neighborhood,
Bath Beach,46
Bay Ridge,83
Bedford Stuyvesant,30
Bensonhurst,36
Bergen Beach,8
Boerum Hill,92
Borough Park,22
Brighton Beach,43
Broadway Junction,25


Lets find out how many unique categories in brooklyn's neighborhood

In [88]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 296 uniques categories.


Run the same above steps for Manhatten

In [89]:
manhattan_venues = getNearbyVenues(names=Manhattan_data['Neighborhood'],
                                   latitudes=Manhattan_data['Latitude'],
                                   longitudes=Manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [91]:
manhattan_venues.groupby('Neighborhood')[['Venue Category']].count()

,Venue Category
Neighborhood,
Battery Park City,66
Carnegie Hill,86
Central Harlem,46
Chelsea,100
Chinatown,100
Civic Center,100
Clinton,100
East Harlem,40
East Village,100


In [92]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 323 uniques categories.


Do the same for queens

In [93]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [94]:
queens_venues.groupby('Neighborhood')[['Venue Category']].count()

,Venue Category
Neighborhood,
Arverne,21
Astoria,100
Astoria Heights,13
Auburndale,20
Bay Terrace,38
Bayside,72
Bayswater,1
Beechhurst,18
Bellaire,14


In [95]:
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 290 uniques categories.


Repeat same steps for Bronx

In [96]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [98]:

bronx_venues.groupby('Neighborhood')[['Venue Category']].count()

,Venue Category
Neighborhood,
Allerton,34
Baychester,21
Bedford Park,38
Belmont,98
Bronxdale,14
Castle Hill,8
City Island,33
Claremont Village,22
Clason Point,15


In [99]:
print('There are {} uniques categories.'.format(len(bronx_venues['Venue Category'].unique())))

There are 186 uniques categories.


Repeat the process for Staten Island

In [100]:
staten_venues = getNearbyVenues(names=staten_data['Neighborhood'],
                                   latitudes=staten_data['Latitude'],
                                   longitudes=staten_data['Longitude']
                                  )

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


In [101]:
staten_venues.groupby('Neighborhood')[['Venue Category']].count()

,Venue Category
Neighborhood,
Annadale,12
Arden Heights,6
Arlington,7
Arrochar,24
Bay Terrace,12
Bloomfield,4
Bulls Head,46
Butler Manor,5
Castleton Corners,17


In [102]:
print('There are {} uniques categories.'.format(len(staten_venues['Venue Category'].unique())))

There are 191 uniques categories.


# Analyze each Neighborhood of each Borough

We will analyze the each neighborhood of each borough of our five areas and the result will be top 10 restaurents

#### Brooklyn

In [103]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column

fixed_columns = [brooklyn_onehot.columns[183]] + list(brooklyn_onehot.columns[:183]) + list(brooklyn_onehot.columns[184:])                                                                    
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Music Venue,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store

In [104]:
brooklyn_onehot.shape

(2869, 296)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [119]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped.head()

,Neighborhood,Music Venue,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store

Manually select all the columns which may have Restaurant features

In [127]:
# this list  is created manually
# selecting only features or categories which are restaurant/eateries
restrology_features = ['Neighborhood', 'American Restaurant', 'Arepa Restaurant', 'Argentinian Restaurant', 'Asian Restaurant', 'BBQ Joint', 'Bagel Shop',
                       'Bakery', 'Bistro', 'Breakfast Spot', 'Bubble Tea Shop', 'Buffet', 'Burger Joint',
                       'Burrito Place', 'Café', 'Cajun / Creole Restaurant', 'Cantonese Restaurant',
                       'Caribbean Restaurant', 'Caucasian Restaurant', 'Chinese Restaurant', 'Coffee Shop',
                       'Creperie', 'Cuban Restaurant', 'Deli / Bodega', 'Dessert Shop', 'Dim Sum Restaurant', 
                       'Diner', 'Donut Shop', 'Dumpling Restaurant', 'Eastern European Restaurant', 'Ethiopian Restaurant', 
                       'Falafel Restaurant', 'Fast Food Restaurant', 'Filipino Restaurant', 'Fish & Chips Shop', 'Food', 
                       'Food & Drink Shop', 'Food Court', 'Food Stand', 'Food Truck', 'French Restaurant', 
                       'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Gastropub', 'German Restaurant', 'Gourmet Shop', 
                       'Greek Restaurant', 'Halal Restaurant', 'Hawaiian Restaurant', 'Health Food Store',  
                       'Hotpot Restaurant', 'Ice Cream Shop', 'Indian Restaurant', 'Israeli Restaurant', 'Italian Restaurant', 
                       'Japanese Restaurant', 'Juice Bar', 'Kebab Restaurant', 'Korean Restaurant', 
                       'Latin American Restaurant',  'Mediterranean Restaurant', 'Mexican Restaurant', 
                       'Middle Eastern Restaurant', 'New American Restaurant', 'Noodle House', 'Pakistani Restaurant', 
                       'Peruvian Restaurant', 'Pie Shop', 'Pizza Place', 'Polish Restaurant', 'Ramen Restaurant', 'Restaurant', 
                       'Russian Restaurant', 'Salad Place', 'Sandwich Place', 'Seafood Restaurant', 'Shabu-Shabu Restaurant', 
                       'Shanghai Restaurant', 'Snack Place', 'South American Restaurant', 'Southern / Soul Food Restaurant', 
                       'Spanish Restaurant', 'Steakhouse', 'Sushi Restaurant', 'Taco Place', 'Taiwanese Restaurant', 
                       'Tapas Restaurant', 'Tea Room', 'Thai Restaurant', 'Tibetan Restaurant', 'Turkish Restaurant', 
                       'Varenyky restaurant', 'Vegetarian / Vegan Restaurant', 'Vietnamese Restaurant', 'Wings Joint']

In [128]:
brooklyn_grouped = brooklyn_grouped[restrology_features]
brooklyn_grouped.head()

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Coffee Shop,Creperie,Cuban Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,German Restaurant,Gourmet Shop,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Health Food Store,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,Pakistani Restaurant,Peruvian Restaurant,Pie Shop,Pizza Place,Polish Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Bath Beach,0.000000,0.0,0.0,0.021739,0.000000,0.021739,0.021739,0.0,0.000000,0.043478,0.0,0.021739,0.0,0.000000,0.0,0.043478,0.0,0.000000,0.043478,0.021739,0.0,0.0,0.000000,0.043478,0.021739,0.0,0.043478,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.021739,0.000000,0.000000,0.021739,0.0,0.0,0.000000,0.021739,0.021739,0.0,0.043478,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.021739,0.0,0.043478,0.0,0.0,0.021739,0.0,0.0,0.021739,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.021739,0.0,0.021739,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.021739,0.0,0.0,0.000000,0.0
1,Bay Ridge,0.036145,0.0,0.0,0.000000,0.000000,0.024096,0.000000,0.0,0.012048,0.000000,0.0,0.000000,0.0,0.024096,0.0,0.000000,0.0,0.012048,0.024096,0.000000,0.0,0.0,0.000000,0.000000,0.012048,0.0,0.000000,0.0,0.0,0.0,0.0,0.012048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.036145,0.000000,0.0,0.0,0.000000,0.024096,0.012048,0.0,0.072289,0.000000,0.012048,0.0,0.0,0.0,0.012048,0.024096,0.012048,0.012048,0.000000,0.0,0.000000,0.0,0.048193,0.0,0.0,0.000000,0.0,0.0,0.024096,0.012048,0.000000,0.0,0.012048,0.0,0.0,0.000000,0.0,0.024096,0.012048,0.0,0.0,0.012048,0.024096,0.0,0.000000,0.0,0.0,0.012048,0.0
2,Bedford Stuyvesant,0.000000,0.0,0.0,0.000000,0.033333,0.033333,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.066667,0.0,0.000000,0.0,0.000000,0.000000,0.100000,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.033333,0.033333,0.033333,0.0,0.0,0.0,0.000000,0.033333,0.000000,0.033333,0.000000,0.0,0.000000,0.0,0.066667,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
3,Bensonhurst,0.000000,0.0,0.0,0.027778,0.000000,0.027778,0.027778,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.111111,0.027778,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.027778,0.055556,0.000000,0.0,0.083333,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.027778,0.0,0.000000,0.0,0.055556,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0

In [129]:
brooklyn_grouped.shape

(70, 95)

Print each neighborhood with top 5 common venues

In [130]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0  Fast Food Restaurant  0.04
1       Bubble Tea Shop  0.04
2          Dessert Shop  0.04
3    Chinese Restaurant  0.04
4  Cantonese Restaurant  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.07
1          Pizza Place  0.05
2  American Restaurant  0.04
3     Greek Restaurant  0.04
4   Mexican Restaurant  0.02


----Bedford Stuyvesant----
                 venue  freq
0          Coffee Shop  0.10
1          Pizza Place  0.07
2                 Café  0.07
3  Japanese Restaurant  0.03
4            BBQ Joint  0.03


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.11
1  Italian Restaurant  0.08
2      Ice Cream Shop  0.06
3    Sushi Restaurant  0.06
4         Pizza Place  0.06


----Bergen Beach----
                 venue  freq
0                 Café  0.12
1  American Restaurant  0.00
2   Russian Restaurant  0.00
3     Ramen Restaurant  0.00
4    Polish Restaurant  0.00


----Boerum Hill

Extract top 10 Reastaurent from 'Brooklyn Heights' neighborhood of brooklyn

In [133]:
venues_num = 10
df_bk = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == 'Brooklyn Heights'].T.reset_index()
df_bk.columns = ['venue','freq']
df_bk = df_bk.iloc[1:]
df_bk['freq'] = df_bk['freq'].astype(float)
df_bk = df_bk.round({'freq': 2})
df_bk['freq'] = df_bk['freq'] * 100
top_bk = pd.DataFrame(df_bk.sort_values('freq', ascending=False).reset_index(drop=True).head(venues_num))

In [134]:
top_bk = top_bk.set_index('venue')

In [136]:
top_bk

,freq
venue,
Deli / Bodega,5.0
Italian Restaurant,4.0
Juice Bar,2.0
Mexican Restaurant,2.0
Coffee Shop,2.0
Thai Restaurant,2.0
Pizza Place,2.0
Diner,2.0
Ice Cream Shop,2.0


Put it into a Pandas Dataframe

Write a function to sort the venues in descending order

In [137]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [138]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pizza Place,Cantonese Restaurant,Italian Restaurant,Fast Food Restaurant,Donut Shop,Dessert Shop,Bubble Tea Shop,Chinese Restaurant,German Restaurant,Ice Cream Shop
1,Bay Ridge,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bagel Shop,Café,Ice Cream Shop,Sushi Restaurant,Chinese Restaurant,Thai Restaurant
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Italian Restaurant,New American Restaurant,Deli / Bodega,Mexican Restaurant,Juice Bar,Japanese Restaurant,Gourmet Shop
3,Bensonhurst,Chinese Restaurant,Italian Restaurant,Donut Shop,Pizza Place,Sushi Restaurant,Ice Cream Shop,Hotpot Restaurant,Coffee Shop,Shabu-Shabu Restaurant,Noodle House
4,Bergen Beach,Café,Wings Joint,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant


### Now do the same above steps for our other 4 places

#### Manhattan

In [139]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Res

In [140]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Res

In [144]:
# this list  is created manually
# selecting only features or categories which are restaurant/eateries
restrology_features2 = ['Neighborhood', 'African Restaurant', 'American Restaurant', 'Arepa Restaurant', 'Argentinian Restaurant',
                        'Argentinian Restaurant', 'Asian Restaurant', 'Australian Restaurant', 'Austrian Restaurant', 'BBQ Joint', 'Bagel Shop',
                        'Bakery', 'Bistro', 'Brazilian Restaurant', 'Breakfast Spot', 'Bubble Tea Shop', 'Burger Joint',
                        'Burrito Place', 'Cafeteria', 'Café', 'Cantonese Restaurant',
                        'Caribbean Restaurant', 'Chinese Restaurant', 'Coffee Shop',
                        'Creperie', 'Cuban Restaurant', 'Cupcake Shop', 'Czech Restaurant', 'Deli / Bodega', 'Dessert Shop', 'Dim Sum Restaurant', 
                        'Diner', 'Donut Shop', 'Dumpling Restaurant', 'Eastern European Restaurant', 'Empanada Restaurant', 'Ethiopian Restaurant', 
                        'Falafel Restaurant', 'Fast Food Restaurant', 'Filipino Restaurant', 'Food & Drink Shop', 'Food Court', 'Food Stand', 'Food Truck', 'French Restaurant', 
                        'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Gastropub', 'German Restaurant', 'Gourmet Shop', 
                        'Greek Restaurant', 'Hawaiian Restaurant', 'Health Food Store', 'Himalayan Restaurant', 'Hot Dog Joint', 
                        'Hotpot Restaurant', 'Ice Cream Shop', 'Indian Restaurant', 'Israeli Restaurant', 'Italian Restaurant', 
                        'Japanese Curry Restaurant', 'Japanese Restaurant', 'Jewish Restaurant', 'Juice Bar', 'Kebab Restaurant', 'Korean Restaurant', 
                        'Kosher Restaurant', 'Latin American Restaurant', 'Lebanese Restaurant', 'Malay Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 
                        'Middle Eastern Restaurant', 'Modern European Restaurant', 'Molecular Gastronomy Restaurant', 'Moroccan Restaurant', 'New American Restaurant',
                        'Noodle House', 'North Indian Restaurant', 'Paella Restaurant',  'Persian Restaurant', 'Peruvian Restaurant',
                        'Pie Shop', 'Pizza Place', 'Ramen Restaurant', 'Restaurant', 'Russian Restaurant', 'Salad Place', 'Sandwich Place', 'Scandinavian Restaurant',
                        'Seafood Restaurant', 'Shanghai Restaurant', 'Snack Place', 'Soba Restaurant', 'Soup Place', 'South Indian Restaurant',
                        'Southern / Soul Food Restaurant', 'Spanish Restaurant', 'Steakhouse', 'Sushi Restaurant', 'Swiss Restaurant', 'Szechuan Restaurant', 'Taco Place',
                        'Taiwanese Restaurant', 'Tapas Restaurant', 'Tea Room', 'Thai Restaurant', 'Turkish Restaurant', 'Vegetarian / Vegan Restaurant', 'Venezuelan Restaurant', 
                        'Vietnamese Restaurant', 'Wings Joint']

In [145]:
manhattan_grouped = manhattan_grouped[restrology_features2]

In [146]:
manhattan_grouped.head()

,Neighborhood,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Creperie,Cuban Restaurant,Cupcake Shop,Czech Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,German Restaurant,Gourmet Shop,Greek Restaurant,Hawaiian Restaurant,Health Food Store,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Pie Shop,Pizza Place,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Battery Park City,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.015152,0.000000,0.000000,0.0,0.0,0.0,0.00,0.030303,0.015152,0.000000,0.000000,0.00,0.000000,0.015152,0.060606,0.0,0.0,0.00,0.0,0.0,0.000000,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.030303,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.030303,0.00,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.015152,0.0,0.015152,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.015152,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.030303,0.000000,0.000000,0.0,0.0,0.015152,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.015152,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.00,0.00,0.0,0.000000,0.0,0.000000,0.0
1,Carnegie Hill,0.000000,0.000000,0.0,0.011628,0.011628,0.00,0.0,0.00,0.000000,0.011628,0.011628,0.0,0.0,0.0,0.00,0.011628,0.000000,0.000000,0.046512,0.00,0.000000,0.011628,0.093023,0.0,0.0,0.00,0.0,0.0,0.000000,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.011628,0.0,0.0,0.000000,0.0,0.011628,0.034884,0.000000,0.0,0.0,0.0,0.011628,0.00,0.0,0.0,0.0,0.011628,0.00,0.00,0.023256,0.00,0.034884,0.0,0.011628,0.0,0.000000,0.0,0.00,0.011628,0.0,0.0,0.00,0.011628,0.011628,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.023256,0.011628,0.011628,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.000000,0.011628,0.0,0.0,0.011628,0.00,0.000000,0.00,0.00,0.0,0.011628,0.0,0.023256,0.0
2,Central Harlem,0.065217,0.043478,0.0,0.000000,0.000000,0.00,0.0,0.00,0.021739,0.021739,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.021739,0.021739,0.00,0.021739,0.043478,0.021739,0.0,0.0,0.00,0.0,0.0,0.021739,0.00,0.00,0.0,0.00,0.0,0.0,0.021739,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.043478,0.021739,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.021739,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.021739,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.043478,0.00,0.00,0.0,0.0,0.0,0.021739,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.021739,0.00,0.

In [147]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
          venue  freq
0   Coffee Shop  0.06
1  Burger Joint  0.03
2  Gourmet Shop  0.03
3    Food Court  0.03
4   Pizza Place  0.03


----Carnegie Hill----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.05
2   French Restaurant  0.03
3  Italian Restaurant  0.03
4         Pizza Place  0.02


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.04
2  American Restaurant  0.04
3    French Restaurant  0.04
4   Seafood Restaurant  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1  American Restaurant  0.04
2               Bakery  0.04
3   Italian Restaurant  0.03
4       Ice Cream Shop  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.08
1               Bakery  0.05
2         Dessert Shop  0.05
3  American Restaurant  0.04
4    Hotpot Restaurant  0.03


----Civic Center----
                 venue  freq
0          Co

In [148]:
venues_num = 10
df_mn = manhattan_grouped[manhattan_grouped['Neighborhood'] == 'Chelsea'].T.reset_index()
df_mn.columns = ['venue','freq']
df_m

n = df_mn.iloc[1:]
df_mn['freq'] = df_mn['freq'].astype(float)
df_mn = df_mn.round({'freq': 2})
df_mn['freq'] = df_mn['freq'] * 100
top_mn = pd.DataFrame(df_mn.sort_values('freq', ascending=False).reset_index(drop=True).head(venues_num))

In [149]:
top_mn = top_mn.set_index('venue')
top_mn

,freq
venue,
Coffee Shop,9.0
American Restaurant,4.0
Bakery,4.0
Italian Restaurant,3.0
Ice Cream Shop,3.0
French Restaurant,2.0
Cupcake Shop,2.0
Japanese Restaurant,2.0
Café,2.0


In [150]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted2 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted2['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted2.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted2.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Gourmet Shop,Burger Joint,Pizza Place,Food Court,Mexican Restaurant,Steakhouse,Burrito Place,Sandwich Place,Italian Restaurant
1,Carnegie Hill,Coffee Shop,Café,French Restaurant,Italian Restaurant,Vietnamese Restaurant,Pizza Place,Indian Restaurant,Gourmet Shop,Mediterranean Restaurant,Kosher Restaurant
2,Central Harlem,African Restaurant,American Restaurant,Seafood Restaurant,Chinese Restaurant,French Restaurant,Fried Chicken Joint,Cafeteria,Caribbean Restaurant,Coffee Shop,Southern / Soul Food Restaurant
3,Chelsea,Coffee Shop,Bakery,American Restaurant,Ice Cream Shop,Italian Restaurant,Cupcake Shop,Café,Japanese Restaurant,French Restaurant,Asian Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Dessert Shop,American Restaurant,Hotpot Restaurant,Noodle House,Ice Cream Shop,Greek Restaurant,Vietnamese Restaurant,Dim Sum Restaurant


#### Queens

In [151]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[170]] + list(queens_onehot.columns[:170]) + list(queens_onehot.columns[171:])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Mediterranean Restaurant,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / G

In [152]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped.head()

,Neighborhood,Mediterranean Restaurant,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / G

In [155]:
# this list  is created manually
# selecting only features or categories which are restaurant/eateries
restrology_features3 = ['Neighborhood', 'Afghan Restaurant', 'American Restaurant', 'Arepa Restaurant',
                        'Argentinian Restaurant', 'Asian Restaurant', 'BBQ Joint', 'Bagel Shop',
                        'Bakery', 'Bistro', 'Brazilian Restaurant', 'Breakfast Spot', 'Bubble Tea Shop', 'Buffet', 'Burger Joint',
                        'Burrito Place', 'Café', 'Cajun / Creole Restaurant', 'Cantonese Restaurant',
                        'Caribbean Restaurant', 'Chinese Restaurant', 'Coffee Shop', 'Colombian Restaurant', 'Comfort Food Restaurant', 
                        'Cuban Restaurant', 'Deli / Bodega', 'Dessert Shop', 'Dim Sum Restaurant', 
                        'Diner', 'Donut Shop', 'Dumpling Restaurant', 'Eastern European Restaurant', 'Empanada Restaurant', 
                        'Falafel Restaurant', 'Fast Food Restaurant', 'Filipino Restaurant', 'Fish & Chips Shop', 'Food & Drink Shop', 'Food Court', 'Food Truck', 'French Restaurant', 
                        'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Gastropub', 'German Restaurant', 'Gluten-free Restaurant', 'Gourmet Shop', 
                        'Greek Restaurant', 'Halal Restaurant', 'Himalayan Restaurant', 'Hot Dog Joint', 
                        'Hotpot Restaurant', 'Ice Cream Shop', 'Indian Restaurant', 'Italian Restaurant', 
                        'Japanese Restaurant', 'Juice Bar', 'Korean Restaurant', 
                        'Kosher Restaurant', 'Latin American Restaurant', 'Malay Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 
                        'Middle Eastern Restaurant', 'New American Restaurant',
                        'Noodle House', 'Peruvian Restaurant',
                        'Pizza Place', 'Polish Restaurant', 'Ramen Restaurant', 'Restaurant', 'Romanian Restaurant', 'Salad Place', 'Sandwich Place', 
                        'Seafood Restaurant', 'Shanghai Restaurant', 'Snack Place', 'South American Restaurant', 
                        'Southern / Soul Food Restaurant', 'Spanish Restaurant', 'Steakhouse', 'Sushi Restaurant', 'Szechuan Restaurant', 'Taco Place',
                        'Taiwanese Restaurant', 'Tapas Restaurant', 'Tea Room', 'Thai Restaurant', 'Tibetan Restaurant', 'Turkish Restaurant', 'Vegetarian / Vegan Restaurant',  
                        'Vietnamese Restaurant']

In [156]:
queens_grouped = queens_grouped[restrology_features3]

In [157]:
queens_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Halal Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,Peruvian Restaurant,Pizza Place,Polish Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Arverne,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.047619,0.047619,0.00,0.0,0.0,0.000000,0.047619,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.047619,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.047619,0.0,0.0,0.047619,0.0,0.00,0.095238,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.00,0.0
1,Astoria,0.0,0.000000,0.0,0.0,0.0,0.01,0.02,0.030000,0.0,0.01,0.0,0.02,0.0,0.010000,0.000000,0.030000,0.01,0.0,0.0,0.010000,0.010000,0.0,0.01,0.0,0.02,0.02,0.0,0.01,0.000000,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.02,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.000000,0.02,0.03,0.0,0.0,0.0,0.0,0.02,0.03,0.020000,0.02,0.01,0.01,0.0,0.01,0.0,0.03,0.01,0.06,0.0,0.00,0.0,0.030000,0.0,0.0,0.010000,0.0,0.01,0.010000,0.04,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0
2,Astoria Heights,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.076923,0.0,0.00,0.0,0.00,0.0,0.076923,0.000000,0.000000,0.00,0.0,0.0,0.076923,0.000000,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.076923,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.076923,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,Auburndale,0.0,0.050000,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.100000,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.05,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,Bay Terrace,0.0,0.052632,0.0,0.0,0.0,0.00,0.00,0.026316,0.0,0.00,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.026316,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.052632,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.026316,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.026316,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.026316,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0


In [158]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
            venue  freq
0  Sandwich Place  0.10
1      Restaurant  0.05
2   Burrito Place  0.05
3      Donut Shop  0.05
4     Pizza Place  0.05


----Astoria----
                       venue  freq
0  Middle Eastern Restaurant  0.06
1         Seafood Restaurant  0.04
2           Greek Restaurant  0.03
3          Indian Restaurant  0.03
4                 Food Truck  0.03


----Astoria Heights----
                venue  freq
0         Pizza Place  0.08
1  Italian Restaurant  0.08
2              Bakery  0.08
3  Chinese Restaurant  0.08
4        Burger Joint  0.08


----Auburndale----
                  venue  freq
0    Italian Restaurant  0.10
1   American Restaurant  0.05
2          Noodle House  0.05
3  Fast Food Restaurant  0.05
4      Sushi Restaurant  0.05


----Bay Terrace----
                    venue  freq
0              Donut Shop  0.05
1     American Restaurant  0.05
2             Pizza Place  0.03
3  Gluten-free Restaurant  0.03
4                  Bakery  0.03


-

In [159]:
venues_num = 10
df_qu = queens_grouped[queens_grouped['Neighborhood'] == 'Elmhurst'].T.reset_index()
df_qu.columns = ['venue','freq']
df_qu = df_qu.iloc[1:]
df_qu['freq'] = df_qu['freq'].astype(float)
df_qu = df_qu.round({'freq': 2})
df_qu['freq'] = df_qu['freq'] * 100
top_qu = pd.DataFrame(df_qu.sort_values('freq', ascending=False).reset_index(drop=True).head(venues_num))

In [160]:
top_qu = top_qu.set_index('venue')
top_qu

,freq
venue,
Thai Restaurant,19.0
Mexican Restaurant,11.0
Vietnamese Restaurant,6.0
Chinese Restaurant,6.0
South American Restaurant,6.0
Bubble Tea Shop,6.0
Sushi Restaurant,3.0
Pizza Place,3.0
Colombian Restaurant,3.0


In [161]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted3 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted3['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted3.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted3.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Sandwich Place,Coffee Shop,Pizza Place,Thai Restaurant,Restaurant,Café,Burrito Place,Donut Shop,Vietnamese Restaurant,Dumpling Restaurant
1,Astoria,Middle Eastern Restaurant,Seafood Restaurant,Food Truck,Greek Restaurant,Indian Restaurant,Bakery,Mediterranean Restaurant,Pizza Place,Café,Food & Drink Shop
2,Astoria Heights,Pizza Place,Burger Joint,Chinese Restaurant,Bakery,Italian Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
3,Auburndale,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Noodle House,American Restaurant,Fried Chicken Joint,Empanada Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant
4,Bay Terrace,American Restaurant,Donut Shop,Gluten-free Restaurant,Pizza Place,Coffee Shop,Bakery,Steakhouse,Falafel Restaurant,Dessert Shop,Dim Sum Restaurant


#### Bronx

In [162]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Medical Center,Medical Supply Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Moving Target,Music Venue,Nightclub,Nightlife Spot,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Post Office,Print Shop,Pub,Public Art,Puerto Rican Restaurant,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Theme Park,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Shop,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [163]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Medical Center,Medical Supply Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Moving Target,Music Venue,Nightclub,Nightlife Spot,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Post Office,Print Shop,Pub,Public Art,Puerto Rican Restaurant,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Theme Park,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Shop,Wings Joint,Women's Store
0,Allerton,0.0,0.0,0.000000,0.000000,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029412,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.029412,0.000000,0.088235,0.0,0.000000,0.0,0.029412,0.000000,0.029412,0.0,0.0,0.058824,0.000000,0.029412,0.000000,0.029412,0.0,0.0,0.0,0.029412,0.000000,0.029412,0.0,0.0,0.029412,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.029412,0.0,0.000000,0.0,0.0,0.029412,0.0,0.000000,0.0,0.029412,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.029412,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.029412,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.029412,0.000000,0.117647,0.058824,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.058824,0.029412,0.0,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Baychester,0.0,0.0,0.000000,0.047619,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.047619,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.047619,0.0,0.000000,0.0,0.000000,0.0

In [166]:
# this list  is created manually
# selecting only features or categories which are restaurant/eateries
restrology_features4 = ['Neighborhood', 'African Restaurant', 'American Restaurant', 'Arepa Restaurant',
                        'Asian Restaurant', 'BBQ Joint', 'Bagel Shop',
                        'Bakery', 'Breakfast Spot', 'Buffet', 'Burger Joint',
                        'Burrito Place', 'Café',
                        'Caribbean Restaurant', 'Chinese Restaurant', 'Coffee Shop', 'Comfort Food Restaurant', 
                        'Deli / Bodega', 'Dessert Shop', 
                        'Diner', 'Donut Shop', 'Eastern European Restaurant',
                        'Fast Food Restaurant', 'Fish & Chips Shop', 'Food & Drink Shop', 'Food Truck', 'French Restaurant', 
                        'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Gourmet Shop', 
                        'Greek Restaurant', 'Ice Cream Shop', 'Indian Restaurant', 'Italian Restaurant', 
                        'Japanese Restaurant', 'Juice Bar',  
                        'Latin American Restaurant', 'Mexican Restaurant', 
                        'Middle Eastern Restaurant', 'Paella Restaurant', 'Peruvian Restaurant',
                        'Pizza Place', 'Restaurant', 'Sandwich Place', 
                        'Seafood Restaurant', 'South American Restaurant', 
                        'Spanish Restaurant', 'Sushi Restaurant', 
                        'Thai Restaurant', 'Vietnamese Restaurant', 'Wings Joint']

In [167]:
bronx_grouped = bronx_grouped[restrology_features4]

In [168]:
bronx_grouped.head()

,Neighborhood,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gourmet Shop,Greek Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Paella Restaurant,Peruvian Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,South American Restaurant,Spanish Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant,Wings Joint
0,Allerton,0.0,0.000000,0.0,0.0,0.0,0.0,0.029412,0.029412,0.0,0.0,0.0,0.000000,0.0,0.088235,0.000000,0.0,0.058824,0.029412,0.000000,0.029412,0.000000,0.029412,0.0,0.000000,0.000000,0.0,0.029412,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.117647,0.0,0.000000,0.0,0.0,0.029412,0.0,0.0,0.0,0.000000
1,Baychester,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.047619,0.000000,0.0,0.000000,0.000000,0.000000,0.095238,0.000000,0.047619,0.0,0.047619,0.000000,0.0,0.047619,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.047619,0.0,0.0,0.0,0.047619,0.0,0.047619,0.0,0.0,0.047619,0.0,0.0,0.0,0.000000
2,Bedford Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.026316,0.000000,0.0,0.0,0.0,0.000000,0.0,0.078947,0.000000,0.0,0.078947,0.000000,0.105263,0.026316,0.000000,0.000000,0.0,0.000000,0.026316,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.026316,0.000000,0.0,0.0,0.078947,0.0,0.0,0.0,0.078947,0.0,0.052632,0.0,0.0,0.052632,0.0,0.0,0.0,0.000000
3,Belmont,0.0,0.010204,0.0,0.0,0.0,0.0,0.051020,0.000000,0.0,0.0,0.0,0.020408,0.0,0.000000,0.020408,0.0,0.071429,0.030612,0.010204,0.030612,0.010204,0.010204,0.0,0.020408,0.000000,0.0,0.010204,0.0,0.010204,0.0,0.010204,0.0,0.183673,0.010204,0.0,0.0,0.020408,0.0,0.0,0.0,0.091837,0.0,0.020408,0.0,0.0,0.020408,0.0,0.0,0.0,0.010204
4,Bronxdale,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.071429,0.0,0.0,0.0,0.000000,0.0,0.071429,0.071429,0.0,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.071429,0.000000,0.0,0.0,0.071429,0.0,0.0,0.0,0.071429,0.0,0.000000,0.0,0.0,0.071429,0.0,0.0,0.0,0.000000


In [169]:
num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                  venue  freq
0           Pizza Place  0.12
1    Chinese Restaurant  0.09
2         Deli / Bodega  0.06
3          Dessert Shop  0.03
4  Fast Food Restaurant  0.03


----Baychester----
                  venue  freq
0            Donut Shop  0.10
1    Chinese Restaurant  0.05
2        Sandwich Place  0.05
3     Food & Drink Shop  0.05
4  Fast Food Restaurant  0.05


----Bedford Park----
                venue  freq
0               Diner  0.11
1       Deli / Bodega  0.08
2  Mexican Restaurant  0.08
3  Chinese Restaurant  0.08
4         Pizza Place  0.08


----Belmont----
                venue  freq
0  Italian Restaurant  0.18
1         Pizza Place  0.09
2       Deli / Bodega  0.07
3              Bakery  0.05
4          Donut Shop  0.03


----Bronxdale----
                venue  freq
0  Chinese Restaurant  0.07
1       Deli / Bodega  0.07
2  Italian Restaurant  0.07
3  Spanish Restaurant  0.07
4      Breakfast Spot  0.07


----Castle Hill----
               

In [170]:
venues_num = 10
df_bx = bronx_grouped[bronx_grouped['Neighborhood'] == 'Belmont'].T.reset_index()
df_bx.columns = ['venue','freq']
df_bx = df_bx.iloc[1:]
df_bx['freq'] = df_bx['freq'].astype(float)
df_bx = df_bx.round({'freq': 2})
df_bx['freq'] = df_bx['freq'] * 100
top_bx = pd.DataFrame(df_bx.sort_values('freq', ascending=False).reset_index(drop=True).head(venues_num))

In [171]:
top_bx = top_bx.set_index('venue')
top_bx

,freq
venue,
Italian Restaurant,18.0
Pizza Place,9.0
Deli / Bodega,7.0
Bakery,5.0
Donut Shop,3.0
Dessert Shop,3.0
Spanish Restaurant,2.0
Mexican Restaurant,2.0
Food & Drink Shop,2.0


In [172]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted4 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted4['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted4.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted4.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Fried Chicken Joint,Spanish Restaurant,Donut Shop,Dessert Shop,Bakery,Breakfast Spot
1,Baychester,Donut Shop,Food & Drink Shop,Spanish Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Mexican Restaurant,Fast Food Restaurant,Chinese Restaurant,Wings Joint
2,Bedford Park,Diner,Mexican Restaurant,Deli / Bodega,Chinese Restaurant,Pizza Place,Spanish Restaurant,Sandwich Place,Bakery,Donut Shop,Italian Restaurant
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Donut Shop,Dessert Shop,Spanish Restaurant,Sandwich Place,Food & Drink Shop,Café
4,Bronxdale,Mexican Restaurant,Eastern European Restaurant,Spanish Restaurant,Deli / Bodega,Pizza Place,Breakfast Spot,Coffee Shop,Chinese Restaurant,Italian Restaurant,Café


#### Staten Island

In [173]:
# one hot encoding
staten_onehot = pd.get_dummies(staten_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
staten_onehot['Neighborhood'] = staten_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [staten_onehot.columns[-1]] + list(staten_onehot.columns[:-1])
staten_onehot = staten_onehot[fixed_columns]

staten_onehot.head()

,Yoga Studio,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Latin American Restaurant,Lawyer,Liquor Store,Locksmith,Mac & Cheese Joint,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Nightlife Spot,Optical Shop,Other Repair Shop,Outdoors & Recreation,Outlet Mall,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Professional & Other Places,Racetrack,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [174]:
staten_grouped = staten_onehot.groupby('Neighborhood').mean().reset_index()
staten_grouped.head()

,Neighborhood,Yoga Studio,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Latin American Restaurant,Lawyer,Liquor Store,Locksmith,Mac & Cheese Joint,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nightclub,Nightlife Spot,Optical Shop,Other Repair Shop,Outdoors & Recreation,Outlet Mall,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Professional & Other Places,Racetrack,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Annadale,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.083333,0.083333,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.000000,0.0,0.166667,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Arden Heights,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.166667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,

In [177]:
# this list  is created manually
# selecting only features or categories which are restaurant/eateries
restrology_features5 = ['Neighborhood', 'American Restaurant', 'Asian Restaurant', 'BBQ Joint', 'Bagel Shop',
                        'Bakery', 'Breakfast Spot', 'Burger Joint',
                        'Cafeteria', 'Café', 'Cajun / Creole Restaurant',
                        'Caribbean Restaurant', 'Chinese Restaurant', 'Coffee Shop',  
                        'Deli / Bodega', 'Dessert Shop', 'Dim Sum Restaurant', 
                        'Diner', 'Donut Shop', 'Eastern European Restaurant', 
                        'Fast Food Restaurant', 'Filipino Restaurant', 'Food & Drink Shop', 'Food Truck', 'French Restaurant', 
                        'Gastropub', 'German Restaurant', 'Gourmet Shop',  
                        'Ice Cream Shop', 'Indian Restaurant', 'Italian Restaurant', 
                        'Japanese Restaurant', 'Juice Bar',  
                        'Mediterranean Restaurant', 'Mexican Restaurant', 
                        'Middle Eastern Restaurant', 
                        'Peruvian Restaurant', 'Pizza Place', 'Restaurant', 'Sandwich Place', 
                        'Smoothie Shop', 'Snack Place',  
                        'Spanish Restaurant', 'Sri Lankan Restaurant', 'Steakhouse', 'Sushi Restaurant', 'Taco Place',
                        'Tapas Restaurant', 'Thai Restaurant', 'Vegetarian / Vegan Restaurant',  
                        'Vietnamese Restaurant', 'Wings Joint']

In [178]:
staten_grouped = staten_grouped[restrology_features5]

In [179]:
staten_grouped.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Fast Food Restaurant,Filipino Restaurant,Food & Drink Shop,Food Truck,French Restaurant,Gastropub,German Restaurant,Gourmet Shop,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Peruvian Restaurant,Pizza Place,Restaurant,Sandwich Place,Smoothie Shop,Snack Place,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Annadale,0.166667,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.083333,0.0,0.0,0.083333,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.166667,0.083333,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Arden Heights,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.166667,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Arlington,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.142857,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Arrochar,0.000000,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.125000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.041667,0.000000,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.041667,0.0,0.041667,0.0,0.041667,0.000000,0.041667,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Bay Terrace,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
num_top_venues = 5

for hood in staten_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = staten_grouped[staten_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Annadale----
                 venue  freq
0  American Restaurant  0.17
1          Pizza Place  0.17
2                Diner  0.08
3           Restaurant  0.08
4        Deli / Bodega  0.08


----Arden Heights----
                venue  freq
0       Deli / Bodega  0.17
1         Pizza Place  0.17
2         Coffee Shop  0.17
3       Smoothie Shop  0.00
4  Italian Restaurant  0.00


----Arlington----
                 venue  freq
0        Deli / Bodega  0.14
1          Coffee Shop  0.14
2        Smoothie Shop  0.00
3   Italian Restaurant  0.00
4  Japanese Restaurant  0.00


----Arrochar----
                      venue  freq
0             Deli / Bodega  0.12
1        Italian Restaurant  0.08
2                Bagel Shop  0.04
3                Food Truck  0.04
4  Mediterranean Restaurant  0.04


----Bay Terrace----
                 venue  freq
0   Italian Restaurant  0.08
1    French Restaurant  0.08
2     Sushi Restaurant  0.08
3           Donut Shop  0.08
4  American Restaurant  0.00


--

In [181]:
venues_num = 10
df_st = staten_grouped[staten_grouped['Neighborhood'] == 'Bulls Head'].T.reset_index()
df_st.columns = ['venue','freq']
df_st = df_st.iloc[1:]
df_st['freq'] = df_st['freq'].astype(float)
df_st = df_st.round({'freq': 2})
df_st['freq'] = df_st['freq'] * 100
top_st = pd.DataFrame(df_st.sort_values('freq', ascending=False).reset_index(drop=True).head(venues_num))

In [182]:
top_st = top_st.set_index('venue')
top_st

,freq
venue,
Pizza Place,11.0
Chinese Restaurant,7.0
Sandwich Place,7.0
Ice Cream Shop,4.0
Deli / Bodega,4.0
Wings Joint,2.0
Donut Shop,2.0
Diner,2.0
Italian Restaurant,2.0


In [183]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted5 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted5['Neighborhood'] = staten_grouped['Neighborhood']

for ind in np.arange(staten_grouped.shape[0]):
    neighborhoods_venues_sorted5.iloc[ind, 1:] = return_most_common_venues(staten_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted5.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,American Restaurant,Pizza Place,Diner,Deli / Bodega,Restaurant,Bagel Shop,Bakery,BBQ Joint,Food & Drink Shop,Filipino Restaurant
1,Arden Heights,Coffee Shop,Deli / Bodega,Pizza Place,Gastropub,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop
2,Arlington,Coffee Shop,Deli / Bodega,Wings Joint,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner
3,Arrochar,Deli / Bodega,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Bagel Shop,Mediterranean Restaurant,Sandwich Place,Food Truck,Filipino Restaurant,Fast Food Restaurant
4,Bay Terrace,Italian Restaurant,Sushi Restaurant,Donut Shop,French Restaurant,Wings Joint,Coffee Shop,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant


# Clustring

Since we have sorted data frames from all boroughs and all negibhorhoods, now we will use K-means clustring

#### Cluster Brooklyn

In [185]:
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 1, 4, 4, 1, 4, 4, 4], dtype=int32)

Now add clusters in our dataset

In [186]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data


brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,4,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bagel Shop,Café,Ice Cream Shop,Sushi Restaurant,Chinese Restaurant,Thai Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Chinese Restaurant,Italian Restaurant,Donut Shop,Pizza Place,Sushi Restaurant,Ice Cream Shop,Hotpot Restaurant,Coffee Shop,Shabu-Shabu Restaurant,Noodle House
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Latin American Restaurant,Mexican Restaurant,Bakery,Pizza Place,Deli / Bodega,Ice Cream Shop,Fried Chicken Joint,Breakfast Spot,Creperie,Italian Restaurant
3,Brooklyn,Greenpoint,40.730201,-73.954241,4,Pizza Place,Coffee Shop,Mexican Restaurant,Deli / Bodega,French Restaurant,Sandwich Place,Restaurant,Tea Room,Bagel Shop,Sushi Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,1,Bakery,Pizza Place,Chinese Restaurant,Italian Restaurant,Diner,Donut Shop,Breakfast Spot,Fish & Chips Shop,Dumpling Restaurant,Eastern European Restaurant


Creatre a Map to visualize the clusters

In [188]:
# create map
map_clusters_bk = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_bk)
       
map_clusters_bk

#### QueensClusters

In [213]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 2, 2, 2], dtype=int32)

In [214]:
# add clustering labels
neighborhoods_venues_sorted3.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data


queens_merged = queens_merged.join(neighborhoods_venues_sorted3.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,0,Middle Eastern Restaurant,Seafood Restaurant,Food Truck,Greek Restaurant,Indian Restaurant,Bakery,Mediterranean Restaurant,Pizza Place,Café,Food & Drink Shop
1,Queens,Woodside,40.746349,-73.901842,0,Bakery,Thai Restaurant,Latin American Restaurant,Filipino Restaurant,American Restaurant,Pizza Place,Donut Shop,Gourmet Shop,Café,Sandwich Place
2,Queens,Jackson Heights,40.751981,-73.882821,0,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Thai Restaurant,Mexican Restaurant,Diner,Spanish Restaurant,Pizza Place,Empanada Restaurant
3,Queens,Elmhurst,40.744049,-73.881656,0,Thai Restaurant,Mexican Restaurant,Vietnamese Restaurant,South American Restaurant,Chinese Restaurant,Bubble Tea Shop,Food Court,Ice Cream Shop,Donut Shop,Malay Restaurant
4,Queens,Howard Beach,40.654225,-73.838138,0,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Japanese Restaurant,Seafood Restaurant,Ice Cream Shop


In [216]:
# create map
map_clusters_qu = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_qu)
       
map_clusters_qu

#### Clustring Bronx

In [218]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 4, 1, 1, 1, 2, 4, 1, 4, 4], dtype=int32)

In [219]:
# add clustering labels
neighborhoods_venues_sorted4.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data


bronx_merged = bronx_merged.join(neighborhoods_venues_sorted4.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1,Donut Shop,Dessert Shop,Deli / Bodega,Sandwich Place,Ice Cream Shop,Pizza Place,Caribbean Restaurant,Wings Joint,Café,Eastern European Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,4,Fast Food Restaurant,Bagel Shop,Restaurant,Pizza Place,Chinese Restaurant,Wings Joint,Caribbean Restaurant,Eastern European Restaurant,Donut Shop,Diner
2,Bronx,Eastchester,40.887556,-73.827806,0,Caribbean Restaurant,Diner,Deli / Bodega,Food & Drink Shop,Juice Bar,Fast Food Restaurant,Donut Shop,Seafood Restaurant,Pizza Place,Chinese Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,4,Wings Joint,Caribbean Restaurant,Fish & Chips Shop,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega,Comfort Food Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,4,Food Truck,Caribbean Restaurant,Fish & Chips Shop,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega,Comfort Food Restaurant


In [221]:

# create map
map_clusters_bx = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_bx)
       
map_clusters_bx

#### Clustering Staten Island

In [222]:
# set number of clusters
kclusters = 5

staten_grouped_clustering = staten_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(staten_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 4, 0, 0, 1, 0, 1, 0], dtype=int32)

In [223]:
# add clustering labels
neighborhoods_venues_sorted5.insert(0, 'Cluster Labels', kmeans.labels_)

staten_merged = staten_data


staten_merged = staten_merged.join(neighborhoods_venues_sorted5.set_index('Neighborhood'), on='Neighborhood')

staten_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,0,Italian Restaurant,American Restaurant,Steakhouse,Donut Shop,Coffee Shop,Snack Place,Burger Joint,Pizza Place,Tapas Restaurant,Breakfast Spot
1,Staten Island,New Brighton,40.640615,-74.087017,0,Deli / Bodega,Chinese Restaurant,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner
2,Staten Island,Stapleton,40.626928,-74.077902,0,Pizza Place,Coffee Shop,Sandwich Place,Café,Fast Food Restaurant,Donut Shop,Diner,Mexican Restaurant,Restaurant,American Restaurant
3,Staten Island,Rosebank,40.615305,-74.069805,4,Mexican Restaurant,Italian Restaurant,Pizza Place,Cajun / Creole Restaurant,Eastern European Restaurant,Ice Cream Shop,Chinese Restaurant,Restaurant,Sandwich Place,Filipino Restaurant
4,Staten Island,West Brighton,40.631879,-74.107182,0,Coffee Shop,Italian Restaurant,Breakfast Spot,Deli / Bodega,Japanese Restaurant,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Diner,Wings Joint


In [225]:
# create map
map_clusters_st = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(staten_merged['Latitude'], staten_merged['Longitude'], staten_merged['Neighborhood'], staten_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_st)
       
map_clusters_st

# Examin each cluster

Now we examine the each cluster to distinguish the categories

#### Brooklyn Cluster 1

In [226]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Midwood,Pizza Place,Ice Cream Shop,Bagel Shop,Bakery,Food,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant


#### Brooklyn Cluster 2

In [227]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,Chinese Restaurant,Italian Restaurant,Donut Shop,Pizza Place,Sushi Restaurant,Ice Cream Shop,Hotpot Restaurant,Coffee Shop,Shabu-Shabu Restaurant,Noodle House
2,Sunset Park,Latin American Restaurant,Mexican Restaurant,Bakery,Pizza Place,Deli / Bodega,Ice Cream Shop,Fried Chicken Joint,Breakfast Spot,Creperie,Italian Restaurant
4,Gravesend,Bakery,Pizza Place,Chinese Restaurant,Italian Restaurant,Diner,Donut Shop,Breakfast Spot,Fish & Chips Shop,Dumpling Restaurant,Eastern European Restaurant
7,Manhattan Terrace,Ice Cream Shop,Pizza Place,Donut Shop,Chinese Restaurant,Coffee Shop,Burger Joint,Steakhouse,Deli / Bodega,Bakery,Bagel Shop
9,Crown Heights,Pizza Place,Café,Bakery,Bagel Shop,Coffee Shop,Sushi Restaurant,Deli / Bodega,Fried Chicken Joint,Burger Joint,Food Truck
14,Brownsville,Fried Chicken Joint,Spanish Restaurant,Burger Joint,Deli / Bodega,Restaurant,Chinese Restaurant,Pizza Place,Food Stand,Food Court,Gastropub
26,East New York,Deli / Bodega,Pizza Place,Fried Chicken Joint,Caribbean Restaurant,Fast Food Restaurant,Filipino Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
27,Starrett City,American Restaurant,Caribbean Restaurant,Donut Shop,Chinese Restaurant,Pizza Place,Fried Chicken Joint,Falafel Restaurant,Diner,German Restaurant,Dumpling Restaurant
32,Coney Island,Deli / Bodega,Fried Chicken Joint,Chinese Restaurant,Caribbean Restaurant,Food Stand,Pizza Place,French Restaurant,Food Truck,Diner,Donut Shop
34,Borough Park,Pizza Place,Café,Fast Food Restaurant,American Restaurant,Eastern European Restaurant,Restaurant,Coffee Shop,Chinese Restaurant,Deli / Bodega,Breakfast Spot


#### Brooklyn Cluster 3

In [228]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Canarsie,Caribbean Restaurant,Asian Restaurant,Thai Restaurant,Food,Wings Joint,Fish & Chips Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
59,Paerdegat Basin,Food,Asian Restaurant,Wings Joint,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant


#### Brooklyn Cluster 4

In [229]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Caribbean Restaurant,Deli / Bodega,Mexican Restaurant,Coffee Shop,Juice Bar,Middle Eastern Restaurant,Donut Shop,Pizza Place,Sandwich Place,Gastropub
10,East Flatbush,Chinese Restaurant,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Food & Drink Shop,Fish & Chips Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
25,Cypress Hills,Latin American Restaurant,Fried Chicken Joint,Donut Shop,Ice Cream Shop,Fast Food Restaurant,Caribbean Restaurant,Food,Mexican Restaurant,Pizza Place,South American Restaurant
29,Flatlands,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Wings Joint,Fish & Chips Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
47,Prospect Park South,Pizza Place,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Donut Shop,Mexican Restaurant,Latin American Restaurant,Taco Place,Sandwich Place,Southern / Soul Food Restaurant
56,Rugby,Caribbean Restaurant,Wings Joint,Food,Diner,Chinese Restaurant,Sandwich Place,Seafood Restaurant,Pizza Place,Fast Food Restaurant,Fried Chicken Joint
57,Remsen Village,Caribbean Restaurant,Fast Food Restaurant,Donut Shop,Fried Chicken Joint,Coffee Shop,Salad Place,Sandwich Place,Breakfast Spot,Café,Wings Joint
69,Erasmus,Caribbean Restaurant,Pizza Place,Fried Chicken Joint,Donut Shop,Food Truck,Sandwich Place,Juice Bar,Fast Food Restaurant,Dim Sum Restaurant,Diner


#### Brooklyn Cluster 5

In [230]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bagel Shop,Café,Ice Cream Shop,Sushi Restaurant,Chinese Restaurant,Thai Restaurant
3,Greenpoint,Pizza Place,Coffee Shop,Mexican Restaurant,Deli / Bodega,French Restaurant,Sandwich Place,Restaurant,Tea Room,Bagel Shop,Sushi Restaurant
5,Brighton Beach,Restaurant,Russian Restaurant,Eastern European Restaurant,Sushi Restaurant,Gourmet Shop,Varenyky restaurant,Fried Chicken Joint,Diner,Donut Shop,Bakery
6,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Sandwich Place,American Restaurant,Buffet,Italian Restaurant,Diner,Pizza Place,Restaurant,Russian Restaurant
11,Kensington,Thai Restaurant,Ice Cream Shop,Pizza Place,Sandwich Place,Japanese Restaurant,Deli / Bodega,Mexican Restaurant,Restaurant,Coffee Shop,Café
12,Windsor Terrace,Diner,American Restaurant,Sushi Restaurant,French Restaurant,Italian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Café,Deli / Bodega,Bagel Shop
13,Prospect Heights,Mexican Restaurant,Thai Restaurant,Bakery,American Restaurant,Southern / Soul Food Restaurant,Gourmet Shop,Ice Cream Shop,Diner,New American Restaurant,Pizza Place
15,Williamsburg,Bagel Shop,Coffee Shop,Burger Joint,Steakhouse,Middle Eastern Restaurant,Deli / Bodega,Pizza Place,Latin American Restaurant,Japanese Restaurant,Italian Restaurant
16,Bushwick,Mexican Restaurant,Coffee Shop,Deli / Bodega,Bakery,Café,Pizza Place,Vegetarian / Vegan Restaurant,Latin American Restaurant,Mediterranean Restaurant,Italian Restaurant
17,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Italian Restaurant,New American Restaurant,Deli / Bodega,Mexican Restaurant,Juice Bar,Japanese Restaurant,Gourmet Shop


#### Queens Cluster 1

In [231]:
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Middle Eastern Restaurant,Seafood Restaurant,Food Truck,Greek Restaurant,Indian Restaurant,Bakery,Mediterranean Restaurant,Pizza Place,Café,Food & Drink Shop
1,Woodside,Bakery,Thai Restaurant,Latin American Restaurant,Filipino Restaurant,American Restaurant,Pizza Place,Donut Shop,Gourmet Shop,Café,Sandwich Place
2,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Thai Restaurant,Mexican Restaurant,Diner,Spanish Restaurant,Pizza Place,Empanada Restaurant
3,Elmhurst,Thai Restaurant,Mexican Restaurant,Vietnamese Restaurant,South American Restaurant,Chinese Restaurant,Bubble Tea Shop,Food Court,Ice Cream Shop,Donut Shop,Malay Restaurant
4,Howard Beach,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Japanese Restaurant,Seafood Restaurant,Ice Cream Shop
6,Forest Hills,Pizza Place,Thai Restaurant,Bakery,Snack Place,Donut Shop,Restaurant,Chinese Restaurant,Bagel Shop,Asian Restaurant,Peruvian Restaurant
7,Kew Gardens,Chinese Restaurant,Pizza Place,Indian Restaurant,Italian Restaurant,Donut Shop,Deli / Bodega,Bubble Tea Shop,Juice Bar,New American Restaurant,Sandwich Place
8,Richmond Hill,Pizza Place,Latin American Restaurant,Indian Restaurant,Deli / Bodega,Caribbean Restaurant,Diner,Sandwich Place,Donut Shop,Spanish Restaurant,Fast Food Restaurant
9,Flushing,Hotpot Restaurant,Korean Restaurant,Bubble Tea Shop,Chinese Restaurant,Asian Restaurant,Café,Bakery,Malay Restaurant,Dim Sum Restaurant,Donut Shop
10,Long Island City,Coffee Shop,Pizza Place,Café,Mexican Restaurant,Donut Shop,Deli / Bodega,Dessert Shop,Chinese Restaurant,Restaurant,Dumpling Restaurant


#### Queens Cluster 2

In [232]:
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Whitestone,Bubble Tea Shop,Deli / Bodega,Vietnamese Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
30,Briarwood,Deli / Bodega,Coffee Shop,Indian Restaurant,Fast Food Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
64,Brookville,Deli / Bodega,Vietnamese Restaurant,Comfort Food Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant


#### Queens Cluster 3

In [233]:
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Corona,Mexican Restaurant,Ice Cream Shop,Deli / Bodega,Chinese Restaurant,Restaurant,Donut Shop,Sandwich Place,Café,South American Restaurant,Empanada Restaurant
11,Sunnyside,Pizza Place,Chinese Restaurant,Deli / Bodega,Bakery,South American Restaurant,Coffee Shop,Latin American Restaurant,Italian Restaurant,Peruvian Restaurant,Restaurant
13,Maspeth,Pizza Place,Diner,Deli / Bodega,Chinese Restaurant,Sandwich Place,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Donut Shop,Dessert Shop
15,Glendale,Pizza Place,Deli / Bodega,Food & Drink Shop,Falafel Restaurant,Cuban Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant
17,Woodhaven,Deli / Bodega,Pizza Place,Bagel Shop,Donut Shop,Latin American Restaurant,Mexican Restaurant,Restaurant,Sandwich Place,Fried Chicken Joint,Thai Restaurant
20,College Point,Deli / Bodega,Pizza Place,Bakery,Mexican Restaurant,Chinese Restaurant,Sandwich Place,Seafood Restaurant,Steakhouse,Latin American Restaurant,Asian Restaurant
24,Little Neck,Korean Restaurant,Chinese Restaurant,Italian Restaurant,Deli / Bodega,Bakery,Coffee Shop,Pizza Place,Cantonese Restaurant,Sandwich Place,Donut Shop
25,Douglaston,Chinese Restaurant,Deli / Bodega,Korean Restaurant,Diner,Greek Restaurant,Donut Shop,Bakery,Sushi Restaurant,Ice Cream Shop,Italian Restaurant
27,Bellerose,Deli / Bodega,Italian Restaurant,Pizza Place,American Restaurant,Diner,Donut Shop,Seafood Restaurant,Empanada Restaurant,Cuban Restaurant,Dessert Shop
28,Kew Gardens Hills,Pizza Place,Sushi Restaurant,Deli / Bodega,Restaurant,Burger Joint,Empanada Restaurant,Cuban Restaurant,Dessert Shop,Dim Sum Restaurant,Diner


#### Queens Cluster 4

In [234]:
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Murray Hill,Korean Restaurant,Coffee Shop,Bakery,Fish & Chips Shop,Pizza Place,Fried Chicken Joint,Japanese Restaurant,Asian Restaurant,BBQ Joint,Dessert Shop


#### Queens Cluster 5

In [235]:
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,South Jamaica,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Bakery,Sandwich Place,Deli / Bodega,French Restaurant,Eastern European Restaurant,Dessert Shop,Dim Sum Restaurant,Gastropub
36,St. Albans,Caribbean Restaurant,Deli / Bodega,Fried Chicken Joint,Chinese Restaurant,Café,Seafood Restaurant,Fast Food Restaurant,Donut Shop,Southern / Soul Food Restaurant,Eastern European Restaurant
39,Cambria Heights,Caribbean Restaurant,Restaurant,Chinese Restaurant,Mexican Restaurant,Bakery,Vietnamese Restaurant,Empanada Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
40,Rosedale,Caribbean Restaurant,Deli / Bodega,Chinese Restaurant,Fried Chicken Joint,Sandwich Place,Empanada Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
59,Laurelton,Caribbean Restaurant,Vietnamese Restaurant,Falafel Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


#### Bronox Cluster 1

In [236]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Eastchester,Caribbean Restaurant,Diner,Deli / Bodega,Food & Drink Shop,Juice Bar,Fast Food Restaurant,Donut Shop,Seafood Restaurant,Pizza Place,Chinese Restaurant
8,Williamsbridge,Deli / Bodega,Restaurant,Caribbean Restaurant,Wings Joint,Café,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop
40,Olinville,Caribbean Restaurant,Deli / Bodega,Fried Chicken Joint,Bakery,Pizza Place,Wings Joint,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner


#### Bronox Cluster 2

In [237]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Donut Shop,Dessert Shop,Deli / Bodega,Sandwich Place,Ice Cream Shop,Pizza Place,Caribbean Restaurant,Wings Joint,Café,Eastern European Restaurant
6,Woodlawn,Deli / Bodega,Pizza Place,Food Truck,Donut Shop,Indian Restaurant,Italian Restaurant,Food & Drink Shop,Bakery,Breakfast Spot,Fast Food Restaurant
7,Norwood,Pizza Place,Chinese Restaurant,Deli / Bodega,Food Truck,Coffee Shop,Caribbean Restaurant,Fast Food Restaurant,Mexican Restaurant,Latin American Restaurant,Restaurant
10,Pelham Parkway,Pizza Place,Frozen Yogurt Shop,Italian Restaurant,Donut Shop,Sushi Restaurant,Deli / Bodega,Sandwich Place,Coffee Shop,Ice Cream Shop,Chinese Restaurant
12,Bedford Park,Diner,Mexican Restaurant,Deli / Bodega,Chinese Restaurant,Pizza Place,Spanish Restaurant,Sandwich Place,Bakery,Donut Shop,Italian Restaurant
13,University Heights,Pizza Place,Bakery,Chinese Restaurant,Wings Joint,Latin American Restaurant,Burrito Place,Deli / Bodega,Donut Shop,Fast Food Restaurant,Fried Chicken Joint
14,Morris Heights,Spanish Restaurant,Deli / Bodega,Pizza Place,Paella Restaurant,Wings Joint,Café,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop
22,Longwood,Latin American Restaurant,Fast Food Restaurant,Donut Shop,Diner,Sandwich Place,Deli / Bodega,Wings Joint,Eastern European Restaurant,Dessert Shop,Comfort Food Restaurant
27,Throgs Neck,Chinese Restaurant,American Restaurant,Asian Restaurant,Pizza Place,Deli / Bodega,Italian Restaurant,Coffee Shop,Wings Joint,Fast Food Restaurant,Eastern European Restaurant
31,Van Nest,Pizza Place,Deli / Bodega,Caribbean Restaurant,Donut Shop,Spanish Restaurant,BBQ Joint,Bakery,Middle Eastern Restaurant,Coffee Shop,Chinese Restaurant


#### Bronox Cluster 3

In [238]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,East Tremont,Pizza Place,Fish & Chips Shop,Donut Shop,Breakfast Spot,Café,Caribbean Restaurant,Fast Food Restaurant,Eastern European Restaurant,Diner,Dessert Shop
19,Melrose,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Deli / Bodega,Burrito Place,Eastern European Restaurant,Dessert Shop,Comfort Food Restaurant
37,Schuylerville,Pizza Place,Mexican Restaurant,Diner,Sandwich Place,Donut Shop,Fast Food Restaurant,American Restaurant,Bagel Shop,BBQ Joint,Eastern European Restaurant
39,Castle Hill,Diner,Pizza Place,Wings Joint,Café,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Dessert Shop,Deli / Bodega,Comfort Food Restaurant


#### Bronox Cluster 4

In [239]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Country Club,Sandwich Place,BBQ Joint,Wings Joint,Caribbean Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega


#### Bronox Cluster 5

In [240]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Fast Food Restaurant,Bagel Shop,Restaurant,Pizza Place,Chinese Restaurant,Wings Joint,Caribbean Restaurant,Eastern European Restaurant,Donut Shop,Diner
3,Fieldston,Wings Joint,Caribbean Restaurant,Fish & Chips Shop,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega,Comfort Food Restaurant
4,Riverdale,Food Truck,Caribbean Restaurant,Fish & Chips Shop,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega,Comfort Food Restaurant
5,Kingsbridge,Pizza Place,Sandwich Place,Bakery,Latin American Restaurant,Mexican Restaurant,Donut Shop,Spanish Restaurant,Fried Chicken Joint,Wings Joint,Caribbean Restaurant
9,Baychester,Donut Shop,Food & Drink Shop,Spanish Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Mexican Restaurant,Fast Food Restaurant,Chinese Restaurant,Wings Joint
11,City Island,Seafood Restaurant,American Restaurant,Café,Spanish Restaurant,Diner,French Restaurant,Italian Restaurant,Pizza Place,Bakery,Deli / Bodega
15,Fordham,Donut Shop,Fast Food Restaurant,Spanish Restaurant,Pizza Place,Diner,Deli / Bodega,Chinese Restaurant,Sandwich Place,Bakery,Latin American Restaurant
17,West Farms,Chinese Restaurant,Donut Shop,Diner,Sandwich Place,Pizza Place,Coffee Shop,Wings Joint,Café,Fast Food Restaurant,Eastern European Restaurant
18,High Bridge,Pizza Place,Chinese Restaurant,Latin American Restaurant,Donut Shop,Asian Restaurant,Seafood Restaurant,Deli / Bodega,Ice Cream Shop,Wings Joint,Caribbean Restaurant
20,Mott Haven,Chinese Restaurant,Donut Shop,Pizza Place,Burger Joint,Spanish Restaurant,Bakery,Peruvian Restaurant,Fish & Chips Shop,Latin American Restaurant,Caribbean Restaurant


#### Staten Island Cluster 1

In [241]:
staten_merged.loc[staten_merged['Cluster Labels'] == 0, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,St. George,Italian Restaurant,American Restaurant,Steakhouse,Donut Shop,Coffee Shop,Snack Place,Burger Joint,Pizza Place,Tapas Restaurant,Breakfast Spot
1,New Brighton,Deli / Bodega,Chinese Restaurant,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner
2,Stapleton,Pizza Place,Coffee Shop,Sandwich Place,Café,Fast Food Restaurant,Donut Shop,Diner,Mexican Restaurant,Restaurant,American Restaurant
4,West Brighton,Coffee Shop,Italian Restaurant,Breakfast Spot,Deli / Bodega,Japanese Restaurant,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Diner,Wings Joint
6,Todt Hill,Wings Joint,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dim Sum Restaurant
7,South Beach,Deli / Bodega,Wings Joint,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner
10,Port Ivory,Wings Joint,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dim Sum Restaurant
13,Travis,Pizza Place,Spanish Restaurant,Café,Deli / Bodega,Wings Joint,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop
15,Oakwood,Wings Joint,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner,Dim Sum Restaurant
19,Woodrow,Pizza Place,Mexican Restaurant,Bagel Shop,Bakery,Sushi Restaurant,Donut Shop,Diner,Italian Restaurant,Juice Bar,Chinese Restaurant


#### Staten Island Cluster 2

In [242]:
staten_merged.loc[staten_merged['Cluster Labels'] == 1, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Port Richmond,Food Truck,Donut Shop,Pizza Place,Wings Joint,Gastropub,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Diner
11,Castleton Corners,Pizza Place,Bagel Shop,Ice Cream Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Wings Joint,Deli / Bodega,Filipino Restaurant,Fast Food Restaurant
12,New Springville,Pizza Place,Chinese Restaurant,Bagel Shop,Mexican Restaurant,Deli / Bodega,Restaurant,Sandwich Place,Donut Shop,Ice Cream Shop,Breakfast Spot
17,Eltingville,Pizza Place,Sushi Restaurant,Fast Food Restaurant,Sandwich Place,Italian Restaurant,Donut Shop,Dessert Shop,Japanese Restaurant,Diner,Gourmet Shop
18,Annadale,American Restaurant,Pizza Place,Diner,Deli / Bodega,Restaurant,Bagel Shop,Bakery,BBQ Joint,Food & Drink Shop,Filipino Restaurant
29,Grasmere,Pizza Place,Bagel Shop,Vegetarian / Vegan Restaurant,Bakery,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Restaurant,Deli / Bodega,Coffee Shop
40,Rossville,Pizza Place,Bagel Shop,American Restaurant,Ice Cream Shop,Bakery,Donut Shop,Deli / Bodega,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant
41,Arden Heights,Coffee Shop,Deli / Bodega,Pizza Place,Gastropub,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop
42,Greenridge,Pizza Place,Asian Restaurant,Bagel Shop,Diner,Wings Joint,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant
43,Heartland Village,Pizza Place,Restaurant,Donut Shop,Coffee Shop,Food Truck,Bakery,Deli / Bodega,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant


#### Staten  Island Cluster 3

In [243]:
staten_merged.loc[staten_merged['Cluster Labels'] == 2, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Willowbrook,Chinese Restaurant,Bagel Shop,Deli / Bodega,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop


#### Staten Island Cluster 4

In [244]:
staten_merged.loc[staten_merged['Cluster Labels'] == 3, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Grymes Hill,Deli / Bodega,Wings Joint,Coffee Shop,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop,Diner


#### Staten Island Cluster 5

In [245]:
staten_merged.loc[staten_merged['Cluster Labels'] == 4, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Rosebank,Mexican Restaurant,Italian Restaurant,Pizza Place,Cajun / Creole Restaurant,Eastern European Restaurant,Ice Cream Shop,Chinese Restaurant,Restaurant,Sandwich Place,Filipino Restaurant
9,Mariner's Harbor,Italian Restaurant,Deli / Bodega,Coffee Shop,Gastropub,Food Truck,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop
14,New Dorp,Italian Restaurant,Pizza Place,Coffee Shop,Chinese Restaurant,Deli / Bodega,Dessert Shop,Sandwich Place,Indian Restaurant,Dim Sum Restaurant,Mexican Restaurant
16,Great Kills,Italian Restaurant,Pizza Place,Cafeteria,Japanese Restaurant,Dessert Shop,Chinese Restaurant,Sandwich Place,Spanish Restaurant,Food & Drink Shop,Bakery
20,Tottenville,Italian Restaurant,Mexican Restaurant,Sandwich Place,Deli / Bodega,Wings Joint,Coffee Shop,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant
21,Tompkinsville,Deli / Bodega,Pizza Place,Café,Gastropub,Italian Restaurant,Mexican Restaurant,Caribbean Restaurant,Sandwich Place,Food Truck,Taco Place
28,Arrochar,Deli / Bodega,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Bagel Shop,Mediterranean Restaurant,Sandwich Place,Food Truck,Filipino Restaurant,Fast Food Restaurant
30,Old Town,Italian Restaurant,American Restaurant,Pizza Place,Donut Shop,Deli / Bodega,Restaurant,Bakery,Breakfast Spot,Dessert Shop,Food & Drink Shop
31,Dongan Hills,Pizza Place,Italian Restaurant,Fast Food Restaurant,Bagel Shop,Eastern European Restaurant,Sushi Restaurant,Ice Cream Shop,Sandwich Place,Deli / Bodega,Chinese Restaurant
34,New Dorp Beach,Italian Restaurant,Deli / Bodega,Restaurant,Pizza Place,Diner,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Eastern European Restaurant,Donut Shop
